# 02. TLC Employees Payroll

Dataset used in this project consist TLC employees (TAXI & LIMOUSINE COMMISSION) with remunerations per fiscal year. After a simple extract from the API and load to Postgres, data is modeled in Postgres. The idea behind it is to compress the data (in a run-length encoding style), keep the data ordered (using an array for changing dimensions, which keeps the order after exploding) and to enable historical analysis without shuffle

### Getting data from API

In [ ]:
import requests
import csv
import time

URL = "https://data.cityofnewyork.us/resource/k397-673e.json"

LIMIT = 100000 
OFFSET = 0
all_rows = []

while True:
    params = {
        "$limit": LIMIT,
        "$offset": OFFSET
    }

    response = requests.get(URL, params=params, timeout=30)
    response.raise_for_status()

    batch = response.json()

    if not batch:
        break

    all_rows.extend(batch)
    OFFSET += LIMIT

    time.sleep(0.2) 


fieldnames = set()
for row in all_rows:
    fieldnames.update(row.keys())

fieldnames = sorted(fieldnames)

with open("nyc_payroll.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(all_rows)


### Postgres upload

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

POSTGRES_USER = "postgres"
POSTGRES_PASSWORD = "postgres"
POSTGRES_HOST = "localhost"
POSTGRES_PORT = "5433"    
POSTGRES_DB = "postgres"

engine_url = f"postgresql+psycopg://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"
engine = create_engine(engine_url)

csv_file = r"\nyc_payroll.csv"

table_name = "nyc_payroll"

print("Wczytywanie pliku CSV")
df = pd.read_csv(csv_file, dtype=str, delimiter=';')

print("Zapisywanie do Postgresa")
df.to_sql(table_name, engine, if_exists='append', index=False)

print(f"Dane z pliku CSV zostały dopisane do tabeli {table_name}")

### Postgres data modeling

First, we create a type for all numeric values that an employee has. They will be stored in an array

In [ ]:
CREATE TYPE stats AS(
fiscal_year INT,
base_salary FLOAT,
regular_hours FLOAT,
regular_gross_paid FLOAT,
ot_hours FLOAT,
total_ot_paid FLOAT,
total_other_pay FLOAT
)

Type is then used in new table, where we input our data

In [ ]:
CREATE TABLE nyc_payroll_tlc (
first_name TEXT,
last_name TEXT,
start_date DATE,
title_description TEXT,
pay_basis TEXT,
stats stats[],
years_since_last_employed INTEGER,
current_FY INT,
PRIMARY KEY (first_name, last_name, current_FY)
)


Now we run cumulative query. It has to be executed fully, for all the years one by one, but it's splitted here for readability

<span style="color: blue;">---FULL QUERY STARTS HERE</span>

In [ ]:
INSERT INTO nyc_payroll_tlc

'yesteday' is is based on the data added previously to the new 'nyc_payroll_tlc ' table, so in the first run, it's empty

In [ ]:
WITH yesterday AS (
SELECT *
FROM nyc_payroll_tlc
WHERE current_FY = '2013' #increase by 1 in each iteration
),

'today' is based on the original tabel that was uploaded to Postgres

In [ ]:
today AS(
SELECT *
FROM nyc_payroll
WHERE agency_name = 'TAXI & LIMOUSINE COMMISSION'
AND fiscal_year = '2014' #increase by 1 in each iteration
)

Now, we define the data. If there is a data today, we take it, otherwise we take yesterday

In [ ]:
SELECT
COALESCE(t.first_name, y.first_name) AS first_name,
COALESCE(t.last_name, y.last_name) AS last_name,
COALESCE(CAST(t.agency_start_date AS DATE), y.start_date) AS start_date,
COALESCE(t.title_description, y.title_description) AS title_description,
COALESCE(t.pay_basis, y.pay_basis) AS pay_basis,



Here is the crucial part. There are 3 scenarios:
- if yesterday's stats are NULL, then we take today (meaning there is no history of a record, it's new)
- if today's stats are there (today is NOT NULL), then we take today but concat it with yesterday, meaning what's already in the table. It enables to keep the history in the same row
- otherwise, so if today is NULL (there are no new records) we keep tracking yesterday's historical data

In [ ]:
CASE 
	WHEN y.stats IS NULL THEN ARRAY[ROW(
t.fiscal_year,
t.base_salary,
t.regular_hours,
t.regular_gross_paid,
t.ot_hours,
t.total_ot_paid,
t.total_other_pay
)::stats]
	WHEN t.fiscal_year IS NOT NULL THEN y.stats || ARRAY[ROW(
t.fiscal_year,
t.base_salary,
t.regular_hours,
t.regular_gross_paid,
t.ot_hours,
t.total_ot_paid,
t.total_other_pay
)::stats]
	ELSE y.stats
	END AS stats,

Additionally, we track number of years since the last record occured and we add a year of the latest incremental load of all data

In [ ]:
CASE 
	WHEN t.fiscal_year IS NOT NULL THEN 0
	ELSE y.years_since_last_employed + 1
	END AS years_since_last_employed,
COALESCE(t.fiscal_year, y.current_FY + 1) AS current_FY


Finally, we join both tables based on first and last name. Wheater there is a value in one of the tables, we will get all data (because of FULL JOIN) without extra columns (because of previous COALESCE)

In [ ]:
FROM today t FULL OUTER JOIN yesterday y
ON t.first_name = y.first_name
AND t.last_name = y.last_name

<span style="color: blue;">---FULL QUERY ENDS HERE</span>

Now we can select * from the new table, based on the last added year (it will hold all the previous values if they exist) and some random name as an example

In [ ]:
SELECT * FROM nyc_payroll_tlc
WHERE last_name = 'AFRIN'
AND first_name = 'SABIHA'
AND current_FY = 2025
ORDER BY first_name, last_name, current_FY

We can see the 'stats' are starting in 2015 and ending in 2022 (based on 'years_since_last_employed' column). That is the cumulative table design

![](images/03_payroll.jpg)

If needed, the table could be UNNESTED later, showing all the stats in separate columns

In [ ]:
WITH unnested  AS(
	SELECT first_name, last_name, title_description,
		UNNEST(stats)::stats AS stats
		FROM nyc_payroll_tlc
		WHERE current_FY = 2025
		AND last_name = 'ABRAMOVICH'
		AND first_name = 'MICHAEL'
		)
		
SELECT first_name, last_name,
	(stats::stats).* 
FROM unnested

![](images/04_payroll.jpg)

If we want to UNNEST everything, the data will keep the order based on names and fiscal_year, which helps avoid re-sorting

![](images/05_payroll.jpg)